<a href="https://colab.research.google.com/github/gzanellato-ia/burrowing_parrot/blob/main/whole_colony_counting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://drive.google.com/uc?export=view&id=1CHoxaz7sslroSHaP8IINZtgp6yuVrKZM' width=200 align='right'>

#**Automatic counting of Burrowing Parrot nests**
##Parrot colony in El Cóndor
##Río Negro Province, Argentina

<img src='https://drive.google.com/uc?export=view&id=1YeDCnNV4zKYsjiz_x08wCWAOPH3NaxxV' width=200 align='right'>

##### Gabriel Zanellato - Juan Masello - Gabriel Pagnossin

#### July 2022

## **Algorithm that executes the counting over the whole colony**

<img src='https://drive.google.com/uc?export=view&id=14Qs7erhsP5T1V8KGpsfxEZuKFMnjHRVX' width=300>


In [1]:
# Copyright: Gabriel Zanellato, 2022
# Based on Lempitsky (2010), Fiaschi (2012), Xie(2018), Ronneberger(2015) and Waithe(2017)

from google.colab import drive
drive.mount('/content/drive/')

BASE_FOLDER = '/content/drive/My Drive/Data_loros/'

Mounted at /content/drive/


In [2]:
from skimage import io
import matplotlib.pyplot as plt
import numpy as np
import cv2
import math

### Importing the images

In [3]:
lista_carpetas = ['Km_1', 'Km_2', 'Km_3', 'Km_4', 'Km_5', 'Km_6', 'Km_7', 'Km_8', 'Km_9', 'Km_10', 'Km_11', 'Km_12',
                  'Km_13', 'Km_14', 'Km_15', 'Km_16', 'Km_17', 'Km_18']


In [4]:
import os

nombres_imagenes_des      = []
nombres_imagenes_des_marc = []

for i in lista_carpetas:
    print(i)
    nombres_imagenes_des.append(os.listdir(BASE_FOLDER + '/Conteo_colonia/por_kilometro/' + i))
    nombres_imagenes_des_marc.append(os.listdir(BASE_FOLDER + '/Conteo_colonia/por_kilometro_marcadas/' + i))

Km_1
Km_2
Km_3
Km_4
Km_5
Km_6
Km_7
Km_8
Km_9
Km_10
Km_11
Km_12
Km_13
Km_14
Km_15
Km_16
Km_17
Km_18


In [5]:
print(nombres_imagenes_des)         # list of 18 lists, each one contains the names of the images of each kilometer of the colony
print(nombres_imagenes_des_marc)    # Colab brings them messy

[['IMG_4434.JPG', 'IMG_4435.JPG', 'IMG_4436.JPG', 'IMG_4438.JPG', 'IMG_4439.JPG', 'IMG_4441.JPG', 'IMG_4440.JPG', 'IMG_4442.JPG', 'IMG_4443.JPG', 'IMG_4446.JPG', 'IMG_4445.JPG', 'IMG_4447.JPG', 'IMG_4448.JPG', 'IMG_4451.JPG', 'IMG_4450.JPG', 'IMG_4453.JPG', 'IMG_4452.JPG', 'IMG_4454.JPG', 'IMG_4455.JPG', 'IMG_4456.JPG', 'IMG_4457.JPG', 'IMG_4458.JPG', 'IMG_4460.JPG', 'IMG_4461.JPG', 'IMG_4462.JPG', 'IMG_4464.JPG', 'IMG_4463.JPG', 'IMG_4466.JPG', 'IMG_4465.JPG'], ['IMG_4420.JPG', 'IMG_4418.JPG', 'IMG_4421.JPG', 'IMG_4423.JPG', 'IMG_4426.JPG', 'IMG_4424.JPG', 'IMG_4428.JPG', 'IMG_4427.JPG', 'IMG_4430.JPG', 'IMG_4431.JPG'], ['IMG_4405.JPG', 'IMG_4406.JPG', 'IMG_4408.JPG', 'IMG_4409.JPG', 'IMG_4410.JPG', 'IMG_4412.JPG', 'IMG_4413.JPG', 'IMG_4415.JPG', 'IMG_4417.JPG', 'IMG_4416.JPG'], ['IMG_4400.JPG', 'IMG_4402.JPG', 'IMG_4401.JPG', 'IMG_4404.JPG', 'IMG_4398.JPG', 'IMG_4399.JPG'], ['IMG_4380.jpg', 'IMG_4381.jpg', 'IMG_4387.JPG', 'IMG_4390.JPG', 'IMG_4389.JPG', 'IMG_4391.JPG', 'IMG_4393.JPG'

In [6]:
# Sort the lists

nombres_imagenes      = nombres_imagenes_des
nombres_imagenes_marc = nombres_imagenes_des_marc

for i in range(18):
  nombres_imagenes[i]      = sorted(nombres_imagenes_des[i])
  nombres_imagenes_marc[i] = sorted(nombres_imagenes_des_marc[i])


print(nombres_imagenes)
print('\n', nombres_imagenes_marc)


[['IMG_4434.JPG', 'IMG_4435.JPG', 'IMG_4436.JPG', 'IMG_4438.JPG', 'IMG_4439.JPG', 'IMG_4440.JPG', 'IMG_4441.JPG', 'IMG_4442.JPG', 'IMG_4443.JPG', 'IMG_4445.JPG', 'IMG_4446.JPG', 'IMG_4447.JPG', 'IMG_4448.JPG', 'IMG_4450.JPG', 'IMG_4451.JPG', 'IMG_4452.JPG', 'IMG_4453.JPG', 'IMG_4454.JPG', 'IMG_4455.JPG', 'IMG_4456.JPG', 'IMG_4457.JPG', 'IMG_4458.JPG', 'IMG_4460.JPG', 'IMG_4461.JPG', 'IMG_4462.JPG', 'IMG_4463.JPG', 'IMG_4464.JPG', 'IMG_4465.JPG', 'IMG_4466.JPG'], ['IMG_4418.JPG', 'IMG_4420.JPG', 'IMG_4421.JPG', 'IMG_4423.JPG', 'IMG_4424.JPG', 'IMG_4426.JPG', 'IMG_4427.JPG', 'IMG_4428.JPG', 'IMG_4430.JPG', 'IMG_4431.JPG'], ['IMG_4405.JPG', 'IMG_4406.JPG', 'IMG_4408.JPG', 'IMG_4409.JPG', 'IMG_4410.JPG', 'IMG_4412.JPG', 'IMG_4413.JPG', 'IMG_4415.JPG', 'IMG_4416.JPG', 'IMG_4417.JPG'], ['IMG_4398.JPG', 'IMG_4399.JPG', 'IMG_4400.JPG', 'IMG_4401.JPG', 'IMG_4402.JPG', 'IMG_4404.JPG'], ['IMG_4380.jpg', 'IMG_4381.jpg', 'IMG_4387.JPG', 'IMG_4389.JPG', 'IMG_4390.JPG', 'IMG_4391.JPG', 'IMG_4393.JPG'

In [7]:
# Count the number of images in each kilometer

km = 0
for i in nombres_imagenes:
    km += 1
    print('Km %i: %i images'%(km, len(i)))

Km 1: 29 images
Km 2: 10 images
Km 3: 10 images
Km 4: 6 images
Km 5: 10 images
Km 6: 40 images
Km 7: 18 images
Km 8: 14 images
Km 9: 10 images
Km 10: 15 images
Km 11: 9 images
Km 12: 6 images
Km 13: 5 images
Km 14: 7 images
Km 15: 4 images
Km 16: 16 images
Km 17: 12 images
Km 18: 1 images


In [8]:
# Import the libraries

from keras import backend as K
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras

2.12.0


In [9]:
# Loss functions

def mae(y_true, y_pred):
    """
    Function to compute the loss of the neural network at the end of each epoch
    """
    return K.mean(abs(y_pred - y_true))

def rmse(y_true, y_pred):
    """
    Function to compute the loss of the neural network at the end of each epoch
    """
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# Metric

def r2(y_true, y_pred):
    """
    Function to define r2 values as metric of the neural network at the end of each epoch
    """
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [10]:
y_train_max = 0.04827504360859346   # from the training algorithm
factor = 255. / y_train_max
factor

5282.23241117088

### Define the model to use

In [11]:
# Define the model for the counting

modelo = 'model_ResUNet_s3_e80.h5'

In [12]:
BASE_FOLDER + modelo

'/content/drive/My Drive/Data_loros/model_ResUNet_s3_e80.h5'

In [15]:
cont = 0

conteo_colonia = []

for i in range(len(lista_carpetas)):    # 18 folders
    print('Sector: %s, %i images \n'%(lista_carpetas[i], len(nombres_imagenes[i])))

    conteo_kilometro = []

    km = []
    km_marc = []

    for j in range(len(nombres_imagenes[i])):   # 222 images in total

        conteo_imagen = []

        print(str(j+1) + ': ' + lista_carpetas[i] + '/' + nombres_imagenes[i][j])
        img      = np.array(io.imread(BASE_FOLDER + '/Conteo_colonia/por_kilometro/' + lista_carpetas[i] + '/' + nombres_imagenes[i][j]))
        img_marc = np.array(io.imread(BASE_FOLDER + '/Conteo_colonia/por_kilometro_marcadas/' + lista_carpetas[i] + '/' + nombres_imagenes_marc[i][j]))

        print('img.shape', img.shape)

        # converting BGR to RGB (Skimage and Open CV work with different channel order)
        # https://stackoverflow.com/questions/55128386/python-opencv-depth-of-image-unsupported-cv-64f
        img_marc_rgb = cv2.cvtColor(img_marc.astype(np.uint8), cv2.COLOR_BGR2RGB)

        # Detect the position of yellow lines
        # https://stackoverflow.com/questions/63462731/about-line-detection-by-using-opencv

        hsv = cv2.cvtColor(img_marc_rgb, cv2.COLOR_BGR2HSV)

        low_yellow = np.array([10, 94, 150])
        up_yellow = np.array([40, 255, 255])
        mask = cv2.inRange(hsv, low_yellow, up_yellow)
        edges = cv2.Canny(mask, 75, 150)

        lines = cv2.HoughLinesP(edges, 1, np.pi/180, 50, maxLineGap=250)

        # not all images have marked vertical yellow lines
        NoneType = type(None)   # https://stackoverflow.com/questions/41928835/how-to-access-the-nonetype-type
                                # type(lines) -> NoteType

        if type(lines) == NoneType:

            lim_izq = 0
            lim_der = img.shape[1]

        else:

            # We eliminate the lines with a slope less than 5 to be left only with the vertical ones
            lines_vert = []
            for line in lines:
                if abs((line[0][3]-line[0][1])/(line[0][2]-line[0][0]))>5:
                    lines_vert.append(line)

            lista_lineas = []
            for line in lines_vert:
                    lista_lineas.append(line[0][0])

            if len(lines_vert) == 0:   # in case the algorithm detects horizontal and not vertical lines (eg. IMG_0985.jpg)

                lim_izq = 0
                lim_der = img.shape[1]

            else:

              lim_izq = min(lista_lineas)
              lim_der = max(lista_lineas)

        print('\n   Horizontal position of vertical lines:')
        print('   left_limit: %d px, rigth_limit: %d px'%(lim_izq, lim_der))

        # WE PATCH!!! #######################################

        ancho = lim_der - lim_izq
        print('   net width:', ancho, 'px\n')

        nro_pasos_hor = ancho // 256
        nro_pasos_ver = img.shape[0] // 256

        # we left the first upper row (sky) and last lower row (beach) unpatched > steps_ver = 9
        # We calculate the height and width of the step
        long_parche = ancho // nro_pasos_hor

        nro_pasos_ver = 9
        lim_sup_parche = 0

        lista_parches = []

        for k in range(nro_pasos_ver):

            lim_sup_parche += 256
            lim_izq_parche = lim_izq

            for l in range(nro_pasos_hor):

                parche         = img[lim_sup_parche:lim_sup_parche+long_parche, lim_izq_parche:lim_izq_parche+long_parche]
                parche_256x256 = cv2.resize(parche, dsize=(256, 256), interpolation=cv2.INTER_CUBIC) # resize to 256x2556

                lista_parches.append(parche_256x256)
                lim_izq_parche += long_parche

        parches = np.array(lista_parches)/255.  # We convert it to a Numpy array
        print('   patches.shape', parches.shape)

        # CALL THE MODEL
        new_model = keras.models.load_model(BASE_FOLDER + modelo, custom_objects={'rmse': rmse, 'r2': r2})

        # PREDICT
        pred  = new_model.predict(parches, batch_size=4)      # dimensions delivered (patches qty., 256, 256, 1)
        pred_ = np.squeeze(pred, axis=3)

        # de-scale the density maps to correct values
        dens_pred_descaled = pred_ / factor

        # COUNT THE OBJECTS #######################################################################################

        for m in range(len(lista_parches)):

            conteo_parche = dens_pred_descaled[m].sum()
            conteo_imagen.append(conteo_parche)

        # total image count
        conteo_imagen_ = sum(conteo_imagen)
        print('   \nTotal image count, %s = %.1f'%(nombres_imagenes[i][j], conteo_imagen_))
        print('____________________________________________________\n')


        with open(BASE_FOLDER + '/Conteo_colonia/resultados_conteo_' + modelo + '.txt', 'a') as f:

            f.write(str(j+1) + ': ' + lista_carpetas[i] + '/' + nombres_imagenes[i][j])
            f.write('\n\n')
            for line_text in range(len(lista_parches)):

                f.write('Patch %d count  ='%(line_text+1) + ' %.2f' %conteo_imagen[line_text])
                f.write('\n')
            f.write('\nTotal image count, %s = %.1f \n'%(nombres_imagenes[i][j], conteo_imagen_))
            f.write('____________________________________________________\n')
            f.write('\n')


        conteo_kilometro.append(conteo_imagen_)

    conteo_kilometro_ = sum(conteo_kilometro)

    print('\nTotal count, sector %s = %.1f \n\n'%(lista_carpetas[i], conteo_kilometro_))

    with open(BASE_FOLDER + '/Conteo_colonia/resultados_conteo_' + modelo + '.txt', 'a') as f:
            f.write('\nTotal count, sector %s = %.1f \n\n'%(lista_carpetas[i], conteo_kilometro_))
            f.write('\n')

    conteo_colonia.append(conteo_kilometro_)

    cont+=1

conteo_colonia_ = sum(conteo_colonia)
print('\n############### TOTAL COLONY COUNT = %.1f NEST ENTRANCES ###############'%(conteo_colonia_))

with open(BASE_FOLDER + '/Conteo_colonia/resultados_conteo_' + modelo + '.txt', 'a') as f:
        f.write('\n############### TOTAL COLONY COUNT = %.1f NEST ENTRANCES ###############'%(conteo_colonia_))



Sector: Km_1, 29 images 

1: Km_1/IMG_4434.JPG
img.shape (3168, 4752, 3)


<ipython-input-15-1fe9f8958ff0>:53: RuntimeWarning: divide by zero encountered in int_scalars
  if abs((line[0][3]-line[0][1])/(line[0][2]-line[0][0]))>5:



   Horizontal position of vertical lines:
   left_limit: 291 px, rigth_limit: 4166 px
   net width: 3875 px

   patches.shape (135, 256, 256, 3)
34/34 [==============================] - 7s 192ms/step
   
Total image count, IMG_4434.JPG = 200.3
____________________________________________________

2: Km_1/IMG_4435.JPG
img.shape (3168, 4752, 3)

   Horizontal position of vertical lines:
   left_limit: 904 px, rigth_limit: 3929 px
   net width: 3025 px

   patches.shape (99, 256, 256, 3)
25/25 [==============================] - 5s 192ms/step
   
Total image count, IMG_4435.JPG = 368.5
____________________________________________________

3: Km_1/IMG_4436.JPG
img.shape (3168, 4752, 3)

   Horizontal position of vertical lines:
   left_limit: 853 px, rigth_limit: 3669 px
   net width: 2816 px

   patches.shape (99, 256, 256, 3)
25/25 [==============================] - 5s 192ms/step
   
Total image count, IMG_4436.JPG = 375.2
____________________________________________________

4: Km_1/IMG

<ipython-input-15-1fe9f8958ff0>:53: RuntimeWarning: invalid value encountered in int_scalars
  if abs((line[0][3]-line[0][1])/(line[0][2]-line[0][0]))>5:



   Horizontal position of vertical lines:
   left_limit: 520 px, rigth_limit: 3664 px
   net width: 3144 px

   patches.shape (108, 256, 256, 3)
27/27 [==============================] - 6s 197ms/step
   
Total image count, IMG_4457.JPG = 126.0
____________________________________________________

22: Km_1/IMG_4458.JPG
img.shape (3168, 4752, 3)

   Horizontal position of vertical lines:
   left_limit: 646 px, rigth_limit: 3759 px
   net width: 3113 px

   patches.shape (108, 256, 256, 3)
27/27 [==============================] - 6s 194ms/step
   
Total image count, IMG_4458.JPG = 213.4
____________________________________________________

23: Km_1/IMG_4460.JPG
img.shape (3168, 4752, 3)

   Horizontal position of vertical lines:
   left_limit: 789 px, rigth_limit: 3728 px
   net width: 2939 px

   patches.shape (99, 256, 256, 3)
25/25 [==============================] - 5s 197ms/step
   
Total image count, IMG_4460.JPG = 376.8
____________________________________________________

24: Km_1